In [1]:
import dlt
import pyarrow.parquet as pq
import fsspec

In [2]:
@dlt.resource(table_name="2022")
def my_df():
    parquet_url = "https://datasets-documentation.s3.eu-west-3.amazonaws.com/stackoverflow/parquet/comments/2022.parquet"
    with fsspec.open(parquet_url, mode="rb") as f:
        table = pq.read_table(f)
        df = table.to_pandas()
        yield df

pipeline = dlt.pipeline(
    pipeline_name="parquet_to_minio",
    destination="filesystem",
    dataset_name="comments",
)

load_info = pipeline.run(
    my_df,
    loader_file_format="parquet",
    write_disposition="replace"
)
print(load_info)

/opt/conda/lib/python3.11/site-packages/dlt/common/configuration/specs/config_providers_context.py:138 DeprecatedImportWarning: The `airflow.operators.python.get_current_context` attribute is deprecated. Please use `'airflow.sdk.get_current_context'`.

/opt/conda/lib/python3.11/site-packages/dlt/common/runtime/exec_info.py:89 DeprecatedImportWarning: The `airflow.operators.python.get_current_context` attribute is deprecated. Please use `'airflow.sdk.get_current_context'`.

Pipeline parquet_to_minio load step completed in 3.04 seconds
1 load package(s) were loaded to destination filesystem and into dataset comments
The filesystem destination used s3://bronze location to store data
Load package 1760209491.8317685 is LOADED and contains no failed jobs


In [4]:
pip install -U pyarrow==15.0.2


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 26.9 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 12.0.1
    Uninstalling pyarrow-12.0.1:
      Successfully uninstalled pyarrow-12.0.1
Note: you may need to restart the kernel to use updated packages.
